In [103]:
import pandas as pd
import json

In [104]:
with open('results.json', 'r') as f:
    results = json.load(f)
with open('XOXNO.json', 'r') as f:
    xoxno = json.load(f)

In [105]:
total = {}
for k, v in xoxno.items():
    total[k] = {
        'xoxno': v,
        'verko': results[k]['verko'],
        'ipfs': results[k]['ipfs'],
        'price': results[k]['price']
    }

In [106]:
with open('total.json', 'w') as f:
    json.dump(total, f)

In [107]:
total_df = None
total_dict = {}

In [108]:
for id in total:
    d = {}
    d['price'] = total[id]['price']
    for k, v in total[id]['ipfs'].items():
        if k != 'attributes':
            d[k] = v
    for item in total[id]['ipfs']['attributes']:
        k, v = item['trait_type'], item['value']
        d[k] = v
    d['collectionName'] = total[id]['verko']['collectionName']
    d['gameName'] = total[id]['verko']['gameData'][0]['gameName']
    for k, v in total[id]['verko']['gameData'][0]['dynamicData'].items():
        if k != 'talents':
            d[k] = v
    for item in total[id]['verko']['gameData'][0]['dynamicData']['talents']:
        k, v = item['name'], item['value']
        d[k] = v
    if total_df is None:
        total_df = pd.DataFrame(d, index=[0])
    else:
        total_df = pd.concat([total_df, pd.DataFrame(d, index=[0])], ignore_index=True)
    total_dict[id] = d

In [109]:
with open('total_dict.json', 'w') as f:
    json.dump(total_dict, f, indent=4)
total_df.to_csv('total_df.csv', index=False)

In [110]:
l = {}
for k, v in total_dict.items():
    for k2, v2 in total[k]['xoxno']['Attributes'].items():
        if v[k2] != v2:
            print(k, k2, v[k2], v2)
    for k2, v2 in total[k]['xoxno']['Game Info'].items():
        if k2 == 'Level':
            k2 = 'level'
        if k2 == 'EarnRate':
            k2 = 'earn_rate'
        try:
            if v[k2] != v2:
                if k not in l:
                    l[k] = []
                l[k].append(k2)
        except KeyError:
            if k not in l:
                l[k] = []
            l[k].append(k2)

In [111]:
for k, v in l.items():
    if v[0] != 'Points':
        print(k, total[k]['xoxno']['Game Info']['Level'], total[k]['verko']['gameData'][0]['dynamicData']['level'])

14211 4 1
4097 4 1
4841 2 1
6037 3 1
7055 4 1
832 2 1
13052 13 3
14782 5 1
4085 9 2
7080 8 2


In [112]:
with open('errors.txt', 'w') as f:
    for item in l:
        f.write(item + '\n')